<a href="https://colab.research.google.com/github/edodoe15/Magistarski_Edin_Kohnic/blob/main/Magistarski.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#LSTM

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, GRU, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

def get_season(month):
    if month in [12, 1, 2]: return 0
    elif month in [3, 4, 5]: return 1
    elif month in [6, 7, 8]: return 2
    else: return 3  # Jesen

def month_sin_cos(month):
    sin_m = np.sin(2 * np.pi * month / 12)
    cos_m = np.cos(2 * np.pi * month / 12)
    return sin_m, cos_m

def create_dataset(dataset, months, years, look_back=120):
    dataX, dataY, monthX, yearX, seasonX, sinMonthX, cosMonthX = [], [], [], [], [], [], []
    for i in range(len(dataset) - look_back - 1):
        dataX.append(dataset[i:(i + look_back), 0])
        dataY.append(dataset[i + look_back, 0])
        monthX.append(months[i + look_back])
        yearX.append(years[i + look_back])
        seasonX.append(get_season(months[i + look_back]))
        sin_m, cos_m = month_sin_cos(months[i + look_back])
        sinMonthX.append(sin_m)
        cosMonthX.append(cos_m)
    return np.array(dataX), np.array(dataY), np.array(monthX), np.array(yearX), np.array(seasonX), np.array(sinMonthX), np.array(cosMonthX)

dataframe = pd.read_csv('data_bih_temp.csv', usecols=['monthly_avg_temp', 'Year', 'Month'])
dataset = dataframe['monthly_avg_temp'].values.astype('float32')
years = dataframe['Year'].values
months = dataframe['Month'].values

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset.reshape(-1, 1))

train_data = dataset[years < 2000]
test_data = dataset[years >= 2000]
train_years = years[years < 2000]
test_years = years[years >= 2000]
train_months = months[years < 2000]
test_months = months[years >= 2000]

look_back = 120
trainX, trainY, trainMonthX, trainYearX, trainSeasonX, trainSinMonthX, trainCosMonthX = create_dataset(train_data, train_months, train_years, look_back)
testX, testY, testMonthX, testYearX, testSeasonX, testSinMonthX, testCosMonthX = create_dataset(test_data, test_months, test_years, look_back)

trainX = np.column_stack((trainX, trainSeasonX, trainSinMonthX, trainCosMonthX))
testX = np.column_stack((testX, testSeasonX, testSinMonthX, testCosMonthX))

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True, input_shape=(1, look_back + 3))))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Bidirectional(GRU(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))

model.fit(trainX, trainY, epochs=200, batch_size=32, verbose=2)

trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

trainPredict = scaler.inverse_transform(trainPredict)
testPredict = scaler.inverse_transform(testPredict)
trainY = scaler.inverse_transform(trainY.reshape(-1, 1))
testY = scaler.inverse_transform(testY.reshape(-1, 1))

trainScore = np.sqrt(mean_squared_error(trainY, trainPredict))
print(f'Train Score: {trainScore:.2f} RMSE')

testScore = np.sqrt(mean_squared_error(testY, testPredict))
print(f'Test Score: {testScore:.2f} RMSE')

trainMAPE = mean_absolute_percentage_error(trainY, trainPredict)
testMAPE = mean_absolute_percentage_error(testY, testPredict)
print(f'Train MAPE: {trainMAPE:.2f}%')
print(f'Test MAPE: {testMAPE:.2f}%')

with open('predikcije_test.txt', 'w') as f:
    f.write("Godina, Mjesec, Stvarna temperatura, Predikcija\n")
    for i in range(len(testY)):
        f.write(f"{testYearX[i]}, {testMonthX[i]}, {testY[i][0]:.2f}, {testPredict[i][0]:.2f}\n")

print("Testne predikcije sačuvane u 'predikcije_test.txt'.")



In [ ]:
# Predikcija budućih temperatura LSTM

future_years = np.arange(2026, 2045 + 1)
future_months = np.tile(np.arange(1, 13), len(future_years))

future_predictions = []

last_input = dataset[-look_back:].reshape(look_back, 1)

for year in future_years:
    for month in range(1, 13):

        season = get_season(month)
        sin_month, cos_month = month_sin_cos(month)

        additional_features = np.array([[season, sin_month, cos_month]])

        input_data = np.concatenate((last_input.flatten(), additional_features.flatten())).reshape(1, 1, look_back + 3)


        predicted_temp = model.predict(input_data, verbose=0)

        predicted_temp = scaler.inverse_transform(predicted_temp.reshape(-1, 1))[0][0]


        future_predictions.append([year, month, predicted_temp])

        last_input = np.roll(last_input, -1)
        last_input[-1, 0] = scaler.transform([[predicted_temp]])[0][0]

with open('predikcije_2026_2045.txt', 'w') as f:
    f.write("Godina, Mjesec, Predikcija temperature\n")
    for pred in future_predictions:
        f.write(f"{pred[0]}, {pred[1]}, {pred[2]:.2f}\n")

print("Predikcije za 2026-2045 sačuvane u 'predikcije_2026_2045.txt'.")


In [ ]:
#LINEARNA REGRESIJA

import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

def get_season(month):
    if month in [12, 1, 2]: return 0
    elif month in [3, 4, 5]: return 1
    elif month in [6, 7, 8]: return 2
    else: return 3

def month_sin_cos(month):
    sin_m = np.sin(2 * np.pi * month / 12)
    cos_m = np.cos(2 * np.pi * month / 12)
    return sin_m, cos_m

def create_dataset(dataset, months, years, look_back=120):
    dataX, dataY, monthX, yearX, seasonX, sinMonthX, cosMonthX = [], [], [], [], [], [], []
    for i in range(len(dataset) - look_back - 1):
        dataX.append(dataset[i:(i + look_back), 0])
        dataY.append(dataset[i + look_back, 0])
        m = months[i + look_back]
        monthX.append(m)
        yearX.append(years[i + look_back])
        seasonX.append(get_season(m))
        sin_m, cos_m = month_sin_cos(m)
        sinMonthX.append(sin_m)
        cosMonthX.append(cos_m)
    return np.array(dataX), np.array(dataY), np.array(monthX), np.array(yearX), np.array(seasonX), np.array(sinMonthX), np.array(cosMonthX)

df = pd.read_csv("data_bih_temp.csv", usecols=["monthly_avg_temp", "Year", "Month"])
dataset = df["monthly_avg_temp"].values.astype("float32").reshape(-1, 1)
months = df["Month"].values
years = df["Year"].values

scaler = MinMaxScaler()
dataset_scaled = scaler.fit_transform(dataset)

look_back = 120
train_data = dataset_scaled[years < 2000]
train_months = months[years < 2000]
train_years = years[years < 2000]

test_data = dataset_scaled[years >= 2000]
test_months = months[years >= 2000]
test_years = years[years >= 2000]

trainX, trainY, trainMonthX, trainYearX, trainSeasonX, trainSinX, trainCosX = create_dataset(train_data, train_months, train_years, look_back)
testX, testY, testMonthX, testYearX, testSeasonX, testSinX, testCosX = create_dataset(test_data, test_months, test_years, look_back)

trainX_full = np.column_stack((trainX, trainSeasonX, trainSinX, trainCosX))
testX_full = np.column_stack((testX, testSeasonX, testSinX, testCosX))

model = Ridge(alpha=100)
model.fit(trainX_full, trainY)

train_pred = model.predict(trainX_full)
test_pred = model.predict(testX_full)

trainY_inv = scaler.inverse_transform(trainY.reshape(-1, 1))
testY_inv = scaler.inverse_transform(testY.reshape(-1, 1))
train_pred_inv = scaler.inverse_transform(train_pred.reshape(-1, 1))
test_pred_inv = scaler.inverse_transform(test_pred.reshape(-1, 1))

train_rmse = np.sqrt(mean_squared_error(trainY_inv, train_pred_inv))
test_rmse = np.sqrt(mean_squared_error(testY_inv, test_pred_inv))
train_mape = mean_absolute_percentage_error(trainY_inv, train_pred_inv)
test_mape = mean_absolute_percentage_error(testY_inv, test_pred_inv)

print(f"Train Score: {train_rmse:.2f} RMSE")
print(f"Test Score: {test_rmse:.2f} RMSE")
print(f"Train MAPE: {train_mape:.2f}%")
print(f"Test MAPE: {test_mape:.2f}%")

with open("linear_test_predikcije.txt", "w") as f:
    f.write("Godina, Mesec, Stvarna temperatura, Predikcija\n")
    for i in range(len(testY)):
        f.write(f"{testYearX[i]}, {testMonthX[i]}, {testY_inv[i][0]:.2f}, {test_pred_inv[i][0]:.2f}\n")

print("Testne predikcije sačuvane u 'linear_test_predikcije.txt'.")



In [ ]:
#Predikcije linearna regresija

future_years = np.arange(2026, 2046)
future_months = np.tile(np.arange(1, 13), len(future_years))
future_years_full = np.repeat(future_years, 12)
future_preds = []

last_input = dataset_scaled[-look_back:].reshape(look_back, 1)

for year, month in zip(future_years_full, future_months):
    season = get_season(month)
    sin_m, cos_m = month_sin_cos(month)

    input_seq = last_input.flatten()
    input_full = np.concatenate((input_seq, [season, sin_m, cos_m])).reshape(1, -1)

    pred_scaled = model.predict(input_full)
    pred_temp = scaler.inverse_transform(pred_scaled.reshape(-1, 1))[0][0]

    future_preds.append([year, month, pred_temp])

    norm_pred = scaler.transform([[pred_temp]])[0][0]
    last_input = np.roll(last_input, -1)
    last_input[-1, 0] = norm_pred

with open("linear_predikcije_2026_2045.txt", "w") as f:
    f.write("Godina, Mjesec, Predikcija temperature\n")
    for pred in future_preds:
        f.write(f"{pred[0]}, {pred[1]}, {pred[2]:.2f}\n")

print("Predikcije za 2026-2045 sačuvane u 'linear_predikcije_2026_2045.txt'.")